# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import json

# Import API key
from api_keys import g_key

if g_key == 'YOUR KEY HERE!':
    print('\033[1;31m-------------')
    print('-------------')
    print('')
    print('NOTE: You need to enter your API keys into the api_keys.py file')
    print('for this script to execute')
    print('')
    print('-------------')    
    print('-------------')

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
print('')
print('\033[1;31m...reading data from ../WeatherPy/output_data/cities_weather.csv')
city_weather_df = pd.read_csv('../WeatherPy/output_data/cities_weather.csv')
print('')
print(f" ---> number of cities in our data = {city_weather_df['Temperature'].count()}")

# let's see about removing some rows
print('')
print(" Is that too many cities to work with?  Enter a number of rows to remove from our data")
#remove_n = input('Number to remove:')
#remove_n = int(remove_n)

try:
        remove_n = int(input("Number to remove: "))
except ValueError:
        remove_n = 0

print(f"Ok, removing {remove_n} cities")

drop_indices = np.random.choice(city_weather_df.index, remove_n, replace=False)
city_weather_df = city_weather_df.drop(drop_indices)
print('')
print(f" Ok, we now have {city_weather_df['Temperature'].count()} cities in our data.")



...reading data from ../WeatherPy/output_data/cities_weather.csv

 ---> number of cities in our data = 913

 Is that too many cities to work with?  Enter a number of rows to remove from our data
Number to remove: 
Ok, removing 0 cities

 ok, we now have 913 cities in our data.


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [79]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)
# Store 'Lat' and 'Lng' into  locations 
locations = city_weather_df[["Lat", "Lng"]].astype(float)
humid_city = city_weather_df['Humidity'].astype(float)



In [80]:
# Create our Heatmap layer
# play with the display
figure_layout = {
    'width': '950px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=humid_city, 
                                 dissipating=True, max_intensity=100,
                                 point_radius = 8)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='500px', padding='1px', width='950px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [85]:
# start with our basic data: city_weather_df and drop the data we don't want
#city_weather_df = pd.read_csv('../WeatherPy/output_data/cities_weather.csv')
city_weather_df
print(f" Number of cities in our data before we sort on our weather conditions = {city_weather_df['Temperature'].count()}")

#  * A max temperature lower than 80 degrees but higher than 70.
city_weather_df = city_weather_df[city_weather_df['Temperature'] < 80]
city_weather_df = city_weather_df[city_weather_df['Temperature'] > 70]
print(f" --> max temp = {city_weather_df['Temperature'].max()}")
print(f" --> min temp = {city_weather_df['Temperature'].min()}")
print(f" Number of cities in our data with the right temp = {city_weather_df['Temperature'].count()}")

#  * Wind speed less than 10 mph.
city_weather_df = city_weather_df[city_weather_df['Wind Speed'] < 10]
print(f" --> max wind speed = {city_weather_df['Wind Speed'].max()}")
print(f" Number of cities in our data with the right wind conditions = {city_weather_df['Temperature'].count()}")

#  * Zero cloudiness.
city_weather_df = city_weather_df[city_weather_df['Cloudiness'] == 0]
print(f" --> max cloudiness = {city_weather_df['Cloudiness'].max()}")
print(f" Number of sunny cities remaining in our data = {city_weather_df['Temperature'].count()}")

#  * Drop any rows with NaN
city_weather_df = city_weather_df.dropna()
print(f" Number of cities remaining in our data after we drop any null values = {city_weather_df['Temperature'].count()}")
print('')
print('\033[1;31m...Let\'s take a look at our list of cities to visit!')
city_weather_df

 Number of cities in our data before we sort on our weather conditions = 612
 --> max temp = 79.99
 --> min temp = 70.07
 Number of cities in our data with the right temp = 140
 --> max wind speed = 9.24
 Number of cities in our data with the right wind conditions = 113
 --> max cloudiness = 0
 Number of sunny cities remaining in our data = 17
 Number of cities remaining in our data after we drop any null values = 17

...Let's take a look at our list of cities to visit!


,City Name,City ID,country,Temperature,Humidity,Cloudiness,Wind Speed,Lat,Lng,Date
94,esmeraldas,3464008,BR,70.99,46,0,1.14,-19.7625,-44.3139,1620253275
119,port said,358619,EG,73.40,68,0,7.18,31.2565,32.2841,1620253286
121,mareeba,2158767,AU,70.07,83,0,1.14,-17.0000,145.4333,1620253286
185,pitimbu,3391889,BR,77.00,88,0,5.75,-7.4706,-34.8086,1620253315
210,caravelas,3466980,BR,73.87,83,0,5.66,-17.7125,-39.2481,1620253326
215,noumea,2139521,NC,75.65,83,0,2.30,-22.2763,166.4572,1620253329
294,cairns,2172797,AU,70.79,83,0,1.14,-16.9167,145.7667,1620253362
397,sao joao da barra,3448903,BR,77.00,83,0,9.22,-21.6403,-41.0511,1620253385
422,voh,2137748,NC,78.39,69,0,1.92,-20.9667,164.7000,1620253417
461,alexandria,361058,EG,71.60,60,0,4.61,31.2156,29.9553,1620253203


In [14]:
# let's see about removing some rows
print('')
print(f" Is {city_weather_df['Temperature'].count()} too many cities to work with?  Enter a number of rows to remove from our data")
print('')

try:
        remove_n = int(input("Number to remove: "))
except ValueError:
        remove_n = 0

print(f"Ok, removing {remove_n} cities")

drop_indices = np.random.choice(city_weather_df.index, remove_n, replace=False)
city_weather_df = city_weather_df.drop(drop_indices)
print('')
print(f" ok, we now have {city_weather_df['Temperature'].count()} cities in our data.")


 Is 912 too many cities to work with?  Enter a number of rows to remove from our data

Number to remove: 2
Ok, removing 2 cities

 ok, we now have 910 cities in our data.


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [87]:
# add our hotel name column
hotel_df = city_weather_df
# fill with an obvious fake hotel name
hotel_df['Hotel Name'] = 'Milliways, Hotel At The End of the Universe'
hotel_df['Lat'] = hotel_df['Lat'].astype(str)
hotel_df['Lng'] = hotel_df['Lng'].astype(str)
hotel_df['coords'] = hotel_df['Lat'] + ', ' + hotel_df['Lng']


hotel_df = hotel_df.rename(columns={'City Name': 'City'})
hotel_df

,City,City ID,country,Temperature,Humidity,Cloudiness,Wind Speed,Lat,Lng,Date,Hotel Name,coords
119,port said,358619,EG,73.40,68,0,7.18,31.2565,32.2841,1620253286,"Milliways, Hotel At The End of the Universe","31.2565, 32.2841"
121,mareeba,2158767,AU,70.07,83,0,1.14,-17.0,145.4333,1620253286,"Milliways, Hotel At The End of the Universe","-17.0, 145.4333"
185,pitimbu,3391889,BR,77.00,88,0,5.75,-7.4706,-34.8086,1620253315,"Milliways, Hotel At The End of the Universe","-7.4706, -34.8086"
215,noumea,2139521,NC,75.65,83,0,2.30,-22.2763,166.4572,1620253329,"Milliways, Hotel At The End of the Universe","-22.2763, 166.4572"
397,sao joao da barra,3448903,BR,77.00,83,0,9.22,-21.6403,-41.0511,1620253385,"Milliways, Hotel At The End of the Universe","-21.6403, -41.0511"
422,voh,2137748,NC,78.39,69,0,1.92,-20.9667,164.7,1620253417,"Milliways, Hotel At The End of the Universe","-20.9667, 164.7"
461,alexandria,361058,EG,71.60,60,0,4.61,31.2156,29.9553,1620253203,"Milliways, Hotel At The End of the Universe","31.2156, 29.9553"
509,porto seguro,3452640,BR,77.00,78,0,3.44,-16.4497,-39.0647,1620253402,"Milliways, Hotel At The End of the Universe","-16.4497, -39.0647"
529,terenos,3446619,BR,78.80,44,0,5.75,-20.4422,-54.8603,1620253465,"Milliways, Hotel At The End of the Universe","-20.4422, -54.8603"
828,conde,3385077,BR,77.00,88,0,5.75,-7.2597,-34.9075,1620253390,"Milliways, Hotel At The End of the Universe","-7.2597, -34.9075"


In [74]:
# let's make a loop for an API call on each city
# make our lat/long ready to go:
locations = hotel_df['coords']
# make some empty lists we'll need later
hotel = []
country = []
city_name = []
marker_locations = []

# our general parameters
# loc and iloc: https://towardsdatascience.com/how-to-use-loc-and-iloc-for-selecting-data-in-pandas-bd09cb4c3d79
target_radius = 5000
target_type = "lodging"
i = 0
print('Now we start pulling our hotels...')
for city_coords in locations:
    params = {
      "location": city_coords,
      "radius": target_radius,
      "type": target_type,
      "key": g_key }   

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # ok, let's go get those hotels!
    
    response = requests.get(base_url, params = params).json()
    i = i + 1
    try:
        hotel_name = response["results"][0]["name"]
        hotel.append(hotel_name)
        print(f"   hotel name = {hotel_name}")
        hotel_df.iloc[i, 10] = hotel_name 
        # let's go ahead and build lists for the map points we're going to draw later
        marker_locations.append(city_coords)
        hotel.append(hotel_name)
        city_name.append(hotel_df.iloc[i, 0])
        country.append(hotel_df.iloc[i, 2])

    except:
        print('----> Something not found, skipping <----')
        pass
    
os.system('say "We have our hotels."') 
print('')
print('Are you getting excited for our vacation?!?!')
    

Now we start pulling our hotels...
   hotel name = Forasteiro Pousada - Hotel
   hotel name = Aracan Hotel
   hotel name = Trinity Plains Tourist Park
   hotel name = Pousada Porto Praia
   hotel name = Pousada dos Navegantes
   hotel name = Mantra Trilogy Cairns
   hotel name = Le gîte du Koniambo
   hotel name = Royal Crown Hotel
   hotel name = Porto Cálem Praia Hotel
   hotel name = Rancho Pitzer
   hotel name = INN CORNER RIVER - PRADO - BAHIA - HOUSING AND HOTEL
----> Something not found, skipping <----
Are you getting excited for our vacation?!?!


In [82]:
# NOTE: Do not change any of the code in this cell
# unless you're a madman!  Aaiieeee!
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# we need our coordinates as floats, so let's convert them and avoid an error message later!
hotel_df['Lat'] = hotel_df['Lat'].astype(float)
hotel_df['Lng'] = hotel_df['Lng'].astype(float)
locations = hotel_df[["Lat", "Lng"]]



In [83]:
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', padding='1px', width='950px'))

In [ ]:
# Add marker layer ontop of heat map


# Display figure
